In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
!ls "/content/drive/My Drive/length of stay"
!cd "/content/drive/My Drive/length of stay"
print(os.getcwd())
os.chdir("/content/drive/My Drive/length of stay")
print(os.getcwd())

In [ ]:
## installing required libraries
# !pip install xgboost
# !pip install lightgbm
# bayesian-optimization
# !pip install scikit-optimize

## Defining required libraries
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score, mean_squared_error
from imblearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras import losses
from keras.layers import Dropout
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import itertools
import keras.backend as K
from data_preprocessing import gendata# Reading tabular data
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


train_X, test_X, train_Y, test_Y = gendata(False)

In [ ]:
############ RESPONSE TO CHALLENGE 0 - Missing values - designing estimator for grid search to remove unwanted cols ###############

# define a new sklearn-supported class to be useable in pipeline
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
#
class remove_m_highlymissed_values(BaseEstimator, ClassifierMixin):
  '''
  An estimator to remove highly noisy missing valued columns that even imputation will not work on them
  '''

 ## Designing constructor
  def __init__(self, m=0):
     self.m = m

 ## designing fit function. Here we only need to find and sort the highly missed ...
 ## columns (cols with highest num of missing vals). THen remove it from output dataframe 
  def fit(self, X, y):
     self.cols_indices_sorted = np.argsort(X.isnull().sum().values)[::-1]
     self.X_ = X.iloc[:,self.cols_indices_sorted[self.m+1:]]
     self.y_ = y
     return self

  def 
  (self, X):
     # Input validation
    #  X = check_array(X)
     self.cols_indices_sorted = np.argsort(X.isnull().sum().values)[::-1]
     return X.iloc[:,self.cols_indices_sorted[self.m+1:]]

  def transform(self, X):
     # Input validation
    #  X = check_array(X)
     self.cols_indices_sorted = np.argsort(X.isnull().sum().values)[::-1]
     return X.iloc[:,self.cols_indices_sorted[self.m+1:]]

In [ ]:

############## RESPONSE TO CHALLENGE 3 -  IMBALANCE CLASSES - Result: Using down sampler to reduce number of samples ###############

np.histogram(train_Y)
# frequency of unmatched is 5 time the freq of matched in the labels. ...
# So unbalanced. To tackle this problem, I use SMOTT and ALYASIN, and undersampler

(array([24709, 24566,  9731, 14123,  3754,  2554,   457,    61,    37,
            8]),
 array([ 1. ,  2.6,  4.2,  5.8,  7.4,  9. , 10.6, 12.2, 13.8, 15.4, 17. ]))

In [ ]:
############## Deal with Preprocessing of numeric and categorical features apart ##############



import numpy as np

# random generators
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

# importing required sklearn tools for pipelining, sampling and missing value replacement
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
# from imblearn.pipeline import Pipeline # as we want downsampler to work, we use imblearn (extend of sklearn.pipeline)
from imblearn.pipeline import Pipeline # as we want downsampler to work, we use imblearn (extend of sklearn.pipeline)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler

# importing evaluators
from sklearn.metrics import f1_score # geometric mean (of prec and rec) always detects model weekness if there is one in either prec or recall, as it is less than least of the two.
from sklearn.metrics import roc_auc_score # area under the curve is a powerful metric for assuring that the model has high true-positives while manifesting fp values in bootstraps or folds

# importing print and plot tools
from pprint import pprint
from matplotlib import pyplot

# importing required classifiers:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier # xgboost should obey the syntax of sklearn to be used in sklearn pipeline class
from lightgbm import LGBMClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# make the random number predictable, i.e. the same in each run. Set constant initial number (seed)
np.random.seed(0) ##TODO : REMOVE

In [ ]:
# xgboost classification pipeline
full_pipline_xgb = Pipeline(
    steps=[
        #('sampler', RandomUnderSampler(random_state=42)),
        ('my_classifier', XGBClassifier(seed=1)) ##TODO: SEED
    ]
) 

# Random forest classification pipeline
full_pipline_rf = Pipeline(
    steps=[
        #('sampler', RandomUnderSampler(random_state=42)),
        ('my_classifier', RandomForestRegressor(
            n_estimators=100))
    ]
)

# lightgbm classification pipeline 
full_pipline_lgbm = Pipeline(
    steps=[
        #('sampler', RandomUnderSampler(random_state=42)),
        ('my_classifier', LGBMClassifier(n_jobs=-1, n_estimators=7000, max_bin= [128],num_leaves= [8],reg_alpha= [1.2],reg_lambda= [1.2],min_data_in_leaf= [50],bagging_fraction= [0.5],learning_rate= [0.001] ))
    ]
)

# SVM classification pipeline 
full_pipline_svm = Pipeline(
    steps=[
        #('sampler', RandomUnderSampler(random_state=42)),
        ('my_classifier', svm.SVC( kernel='poly', gamma=0.1) )
    ]
)

# MLP classification pipeline 
full_pipline_mlp = Pipeline(
    steps=[
        #('sampler', RandomUnderSampler(random_state=42)),
        ('my_classifier', MLPClassifier(max_iter=100) )
    ]
)


A. Grid Search

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
import pickle
# XGBOOST hyperparameter search space
param_grid_xgb = {
    'my_classifier__n_estimators': list(range(50,145,5)),  # num of estimators in xgboost is not usully set more than 100 ## `__` denotes attribute 
    'my_classifier__max_depth':list(range(5,10)),  # maximum depth of each tree is set to 20
    'sampler__sampling_strategy':[0.9]
}

# Random forest hyperparameter search space
param_grid_rf = {
                 'my_classifier__n_estimators': list(range(50,120)),  # num of estimators in xgboost is not usully set more than 100 ## `__` denotes attribute 
                 'my_classifier__max_features' : ['auto'], # In random forest, tricks to determine the maximum number of features
                 'my_classifier__max_depth': [None], # In random forest, maximum depth can be either set automatically or usually very small (eg 3 or 5)
                 'my_classifier__min_impurity_decrease': [0.0], # in random forests, minimum impurity to keep on dissecting branches are usually set below 0.5
                #  'sampler__sampling_strategy':(0.01*np.asarray(list(range(75,85)))).tolist(),#[0.75,0.8,0.85,0.9,0.95],
                 'my_classifier__bootstrap':[True,False]
}

# lightGBM hyperparameter search space
param_grid_lgbm ={
                  'my_classifier__num_leaves': list(range(20,30)), # in gradient boost, the number of leaves should be low, but in case of high num of featres, I prefer to check out the higher ones also
                  'my_classifier__min_child_samples': list(range(100,500,100)),  # # minimum samples per each model
                  'my_classifier__min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4], # importance weight of each model 
                  'my_classifier__subsample': list(np.arange(0.2,1,0.1)) , # subsampling probability of instances
                  'my_classifier__colsample_bytree': list(np.arange(0.4,1,0.1)), # sampling probability of column
                  'my_classifier__reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100], # special parameters suitable to tune
                  'my_classifier__reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100], # special parameters suitable to tune
                  # 'sampler__sampling_strategy':(0.01*np.asarray(list(range(75,85)))).tolist() 
                  
}

# SVM hyperparameter search space
param_grid_svm = {
                  'my_classifier__kernel':['poly'],
                  'my_classifier__C': list(range(9,12)), # is the parameter for the soft margin cost function, which controls the influence of each individual support vector. It is usually selected low not to provoke underfitting
                  'my_classifier__gamma':  np.arange(1e-5,1e-4,1e-5) , # weight of each input in RBF kernel mode. I chose a wide range of values for tuning
                  'my_classifier__class_weight':[ None], #  to balance number of instances per each class
                 # 'sampler__sampling_strategy':(0.01*np.asarray(list(range(83,93)))).tolist() 
}

# MLP hyperparameter search space
param_grid_mlp = {
    'my_classifier__hidden_layer_sizes': [(10,30,10) ],
    'my_classifier__alpha': 0.001*np.asarray(range(5,1000,5)),
    'my_classifier__learning_rate': ['constant','adaptive'],
    #'sampler__sampling_strategy':[ 0.8]

}

## Initialize GridSearchCV class to tune hyperparameters 
classifiers_names=['xgb','rf','lgbm','svm','mlp'] 
case_study=1  
best_scores1,bestparams1=[],[]
for iii,(full_pipeline,param_grid) in enumerate(list(zip([full_pipline_xgb,full_pipline_rf,full_pipline_lgbm,full_pipline_svm,full_pipline_mlp],[param_grid_xgb,param_grid_rf,param_grid_lgbm,param_grid_svm,param_grid_mlp]))):
  if iii in [0]:
    continue
  grid_search = GridSearchCV(
      full_pipeline, param_grid, cv=5, verbose=3, n_jobs=2, 
      )

  # run the grid search object
  grid_search.fit(train_X,train_Y)
  val_scores = cross_val_score(grid_search, test_X, test_Y, cv=5)

  best_scores1.append(grid_search.best_score_)
  best_scores1.append(val_scores.mean())
  bestparams1.append(grid_search.best_params_)
  print("val_score: %0.2f accuracy with a standard deviation of %0.2f" % (val_scores.mean(), val_scores.std()))
  print('%d___%f___'%(case_study,float(grid_search.best_score_)) + classifiers_names[iii]+ ': best score {}'.format(grid_search.best_score_))
  print('%d___%f___'%(case_study,float(grid_search.best_score_)) + classifiers_names[iii]+': best params {}'.format(grid_search.best_params_))

  pickle.dump(grid_search,open('%d.pkl'%(iii),'wb'))


Grid Result:


---


val_score: 0.18 accuracy with a standard deviation of 0.00

1___0.134975___xgb: best score 0.134975

1___0.134975___xgb: best params {'my_classifier__max_depth': 5, 'my_classifier__n_estimators': 10}


---


val_score: -0.91 accuracy with a standard deviation of 0.45

1___-0.233610___rf: best score -0.23360989928754777

1___-0.233610___rf: best params {'my_classifier__bootstrap': True, 'my_classifier__max_depth': None, 'my_classifier__max_features': 'auto', 'my_classifier__min_impurity_decrease': 0.0, 'my_classifier__n_estimators': 140}


---



val_score: 0.18 accuracy with a standard deviation of 0.00

1___0.180613___lgbm: best score 0.1806125

1___0.180613___lgbm: best params {'my_classifier__colsample_bytree': 0.5, 'my_classifier__min_child_samples': 300, 'my_classifier__min_child_weight': 0.001, 'my_classifier__num_leaves': 10, 'my_classifier__reg_alpha': 10, 'my_classifier__reg_lambda': 0, 'my_classifier__subsample': 0.5}


---



val_score: 0.10 accuracy with a standard deviation of 0.01

1___0.165325___svm: best score 0.165325

1___0.165325___svm: best params {'my_classifier__C': 9, 'my_classifier__class_weight': None, 'my_classifier__gamma': 1e-05, 'my_classifier__kernel': 'poly'}


---



val_score: 0.05 accuracy with a standard deviation of 0.04

1___0.069013___mlp: best score 0.0690125

1___0.069013___mlp: best params {'my_classifier__alpha': 0.92, 'my_classifier__hidden_layer_sizes': (10, 30, 10), 'my_classifier__learning_rate': 'adaptive'}


---




P.S. This is the result with "sampler". However, after I comments "sampler" in each model, results are not shown up until the program killed itself after 8hrs



B. Random Search


In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
import pickle
# XGBOOST hyperparameter search space
param_rndsrch_xgb = {
    'my_classifier__n_estimators': [100],  # num of estimators in xgboost is not usully set more than 100 ## `__` denotes attribute 
    'my_classifier__max_depth':[50] # maximum depth of each tree is set to 20
}

# Random forest hyperparameter search space
param_rndsrch_rf = {
                 'my_classifier__max_features' : ['auto'], #['auto''sqrt', 'log2'], # In random forest, tricks to determine the maximum number of features
                 'my_classifier__max_depth': [None], #[None] In random forest, maximum depth can be either set automatically or usually very small (eg 3 or 5)
                 'my_classifier__min_impurity_decrease': [1e-07] # in random forests, minimum impurity to keep on dissecting branches are usually set below 0.5
}

# lightGBM hyperparameter search space ##TODO: NOT CHECKED  ERROR NAASHENA MIDE
param_rndsrch_lgbm ={
                  'my_classifier__num_leaves': [10], # in gradient boost, the number of leaves should be low, but in case of high num of featres, I prefer to check out the higher ones also
                  'my_classifier__min_child_samples': [10],  # # minimum samples per each model
                  'my_classifier__min_child_weight': [0.00], # importance weight of each model 
                  'my_classifier__subsample': [0.6], # subsampling probability of instances
                  'my_classifier__colsample_bytree': [0.5], # sampling probability of column
                  'my_classifier__reg_alpha': [50], # special parameters suitable to tune
                  'my_classifier__reg_lambda': [10], # special parameters suitable to tune
}


# SVM hyperparameter search space
param_rndsrch_svm = {
                  'my_classifier__C': [0.01,0.00001], # is the parameter for the soft margin cost function, which controls the influence of each individual support vector. It is usually selected low not to provoke underfitting
                  'my_classifier__gamma': [1,0.01, 0.001], # weight of each input in RBF kernel mode. I chose a wide range of values for tuning
                  'my_classifier__class_weight':['balanced',None], #  to balance number of instances per each class
}

# MLP hyperparameter search space
param_rndsrch_mlp = {
    'my_classifier__hidden_layer_sizes': [(34,32,32,32,17)],#[(100,),(1000,),(50,),(70,),(200,),(50,60)],
    'my_classifier__alpha': [0.001],#[0.0001,0.001,0.01,0.15, 0.1,0.06],
    'my_classifier__learning_rate': ['constant','adaptive']

}



## Initialize RandomizedSearchCV class to tune hyperparameters 
classifiers_names=['xgb','rf','lgbm','svm','mlp'] 
case_study=2 
best_scores2,bestparams2=[],[]
#for iii,(full_pipeline,param_rndsrch) in enumerate(list(zip([full_pipline_xgb,full_pipline_rf,full_pipline_lgbm,full_pipline_svm,full_pipline_mlp],[param_rndsrch_xgb,param_rndsrch_rf,param_rndsrch_lgbm,param_rndsrch_svm,param_rndsrch_mlp]))):
for iii,(full_pipeline,param_rndsrch) in enumerate(list(zip([full_pipline_xgb],[param_rndsrch_xgb]))):

  if iii<=-1:
    best_scores2.append(None)
    bestparams2.append(None)
    continue

  random_search = RandomizedSearchCV(full_pipeline, param_distributions=param_rndsrch, n_iter=500, cv=5, verbose=3, n_jobs=2)

  # run the garch.predict(data_test)[:,1]rid search object
  random_search.fit(train_X,train_Y)
  val_scores = cross_val_score(random_search, test_X, test_Y, cv=5)

  best_scores2.append(random_search.best_score_)
  best_scores2.append(val_scores.mean())
  bestparams2.append(random_search.best_params_)
  print('-----------------------------------------------')
  print("val_score: %0.2f accuracy with a standard deviation of %0.2f" % (val_scores.mean(), val_scores.std()))
  print('%d___%f___'%(case_study,float(random_search.best_score_)) + classifiers_names[iii]+ ': best score {}'.format(random_search.best_score_))
  print('%d___%f___'%(case_study,float(random_search.best_score_)) + classifiers_names[iii]+': best params {}'.format(random_search.best_params_))
  print('-----------------------------------------------')

  #pickle.dump(random_search,open('%d___%d.pkl'%(iii,case_study),'wb')) # profiling the results in a formatted way

Random Search Result:

---



---




---



**XGB**:
val_score: 0.50 accuracy with a standard deviation of 0.01

2___0.496200___xgb: best score 0.4962

2___0.496200___xgb: best params {'my_classifier__n_estimators': 1, 'my_classifier__max_depth': 5}

---


val_score: 0.56 accuracy with a standard deviation of 0.01

2___0.581950___xgb: best score 0.5819500000000001

2___0.581950___xgb: best params {'my_classifier__n_estimators': 3, 'my_classifier__max_depth': 13}

---


val_score: 0.60 accuracy with a standard deviation of 0.01

2___0.648925___xgb: best score 0.648925

2___0.648925___xgb: best params {'my_classifier__n_estimators': 10, 'my_classifier__max_depth': 50}


---


**val_score: 0.62 accuracy with a standard deviation of 0.01**

**2___0.667450___xgb: best score 0.66745**

**2___0.667450___xgb: best params {'my_classifier__n_estimators': 100, 'my_classifier__max_depth': 20})**


---




---




---



**RF**
val_score: 0.72 accuracy with a standard deviation of 0.00

2___0.723599___rf: best score 0.723599155416591

2___0.723599___rf: best params {'my_classifier__min_impurity_decrease': 0.001, 'my_classifier__max_features': 'auto', 'my_classifier__max_depth': 4}


---


val_score: 0.82 accuracy with a standard deviation of 0.00

2___0.800333___rf: best score 0.8003326578235643

2___0.800333___rf: best params {'my_classifier__min_impurity_decrease': 0.005, 'my_classifier__max_features': 'auto', 'my_classifier__max_depth': None}


---



val_score: 0.90 accuracy with a standard deviation of 0.00

2___0.920077___rf: best score 0.9200768975273762

2___0.920077___rf: best params {'my_classifier__min_impurity_decrease': 1e-06, 'my_classifier__max_features': 'auto', 'my_classifier__max_depth': None}


---



**val_score: 0.90 accuracy with a standard deviation of 0.00**

**2___0.920111___rf: best score 0.9201108383565064**

**2___0.920111___rf: best params {'my_classifier__min_impurity_decrease': 1e-07, 'my_classifier__max_features': 'auto', 'my_classifier__max_depth': None}**


---



---





---



**LSTM**


---



---





---




**SVM**

---



---





---




**MLP**：
val_score: 0.60 accuracy with a standard deviation of 0.00

2___0.618437___mlp: best score 0.6184375

2___0.618437___mlp: best params {'my_classifier__learning_rate': 'constant', 'my_classifier__hidden_layer_sizes': (10, 40, 30), 'my_classifier__alpha': 0.001}


---



val_score: 0.60 accuracy with a standard deviation of 0.00

2___0.628713___mlp: best score 0.6287125

2___0.628713___mlp: best params {'my_classifier__learning_rate': 'constant', 'my_classifier__hidden_layer_sizes': (200,), 'my_classifier__alpha': 0.001}


---



val_score: 0.60 accuracy with a standard deviation of 0.00

2___0.626100___mlp: best score 0.6261

2___0.626100___mlp: best params {'my_classifier__learning_rate': 'adaptive', 'my_classifier__hidden_layer_sizes': (50,), 'my_classifier__alpha': 0.001}


---



val_score: 0.60 accuracy with a standard deviation of 0.01

2___0.628175___mlp: best score 0.628175

2___0.628175___mlp: best params {'my_classifier__learning_rate': 'adaptive', 'my_classifier__hidden_layer_sizes': (100,), 'my_classifier__alpha': 0.01}


---



val_score: 0.61 accuracy with a standard deviation of 0.00

2___0.627062___mlp: best score 0.6270625

2___0.627062___mlp: best params {'my_classifier__learning_rate': 'constant', 'my_classifier__hidden_layer_sizes': (100,), 'my_classifier__alpha': 0.0001}


---



2___0.628063___mlp: best score 0.6280625000000001

2___0.628063___mlp: best params {'my_classifier__learning_rate': 'adaptive', 'my_classifier__hidden_layer_sizes': (100,), 'my_classifier__alpha': 0.001}


---



**val_score: 0.62 accuracy with a standard deviation of 0.01**

**2___0.686600___mlp: best score 0.6866**

**2___0.686600___mlp: best params {'my_classifier__learning_rate': 'constant', 'my_classifier__hidden_layer_sizes': (34, 68, 17), 'my_classifier__alpha': 0.001}**


---





C. Bayesian

In [ ]:
!pip install scikit-optimize
from google.colab import drive
drive.mount('/content/drive/')
import warnings
warnings.filterwarnings("ignore")

import pickle
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score

# XGBOOST hyperparameter search space
param_bayesS_xgb = {
    'my_classifier__n_estimators': [70],  # num of estimators in xgboost is not usully set more than 100 ## `__` denotes attribute 
    'my_classifier__max_depth': [15], # maximum depth of each tree is set to 20

    #'my_classifier__eta': [0.1,0.001,0.0001,0.5],
    #'my_classifier__gamma': [0,10,20,100],
    #'my_classifier__min_child_weight':[1,10,20],
    #'my_classifier__subsample': [0.7,0.8,0.9],
    #'my_classifier__colsample_bytree': [0.8,1,0.2,0.1],
}

# Random forest hyperparameter search space
param_bayesS_rf = {
                 'my_classifier__max_features': ['auto', 'sqrt', 'log2'], # In random forest, tricks to determine the maximum number of features
                 'my_classifier__max_depth': [None,5], # In random forest, maximum depth can be either set automatically or usually very small (eg 3 or 5)
                 'my_classifier__min_impurity_decrease': [10.0], # in random forests, minimum impurity to keep on dissecting branches are usually set below 0.5
                 #'my_classifier__min_samples_split': [10],
}

# lightGBM hyperparameter search space
param_bayesS_lgbm ={
                  'my_classifier__num_leaves': [100], # in gradient boost, the number of leaves should be low, but in case of high num of featres, I prefer to check out the higher ones also
                  'my_classifier__min_child_samples': [400],  # # minimum samples per each model
                  'my_classifier__min_child_weight': [1], # importance weight of each model 
                  'my_classifier__subsample': [0.8], # subsampling probability of instances
                  'my_classifier__colsample_bytree': [0.6], # sampling probability of column
                  'my_classifier__reg_alpha': [0.01], # special parameters suitable to tune
                  'my_classifier__reg_lambda': [0.001], # special parameters suitable to tune
}


# SVM hyperparameter search space
param_bayesS_svm = {
                  'my_classifier__C': [20.0], # is the parameter for the soft margin cost function, which controls the influence of each individual support vector. It is usually selected low not to provoke underfitting
                  'my_classifier__gamma': ['scale','auto'], #'auto'# weight of each input in RBF kernel mode. I chose a wide range of values for tuning
                  'my_classifier__class_weight':[None], #  to balance number of instances per each class
                  'my_classifier__degree': [10],
                  'my_classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

# MLP hyperparameter search space
param_bayesS_mlp = {
    'my_classifier__hidden_layer_sizes': [10,20],
    'my_classifier__alpha': [0.1],
    'my_classifier__learning_rate': ['constant','adaptive','invscaling'],
    'my_classifier__solver': ['sgd','adam','lbfgs']

}



## Initialize RandomizedSearchCV class to tune hyperparameters 
#classifiers_names=['xgb','rf','lgbm','svm','mlp']
classifiers_names=['xgb','rf','mlp']

case_study=3 
best_scores3,bestparams3=[],[]
#for iii,(full_pipeline,param_bayesS) in enumerate(list(zip([full_pipline_xgb,full_pipline_rf,full_pipline_lgbm,full_pipline_svm,full_pipline_mlp],[param_bayesS_xgb,param_bayesS_rf,param_bayesS_lgbm,param_bayesS_svm,param_bayesS_mlp]))):
for iii,(full_pipeline,param_bayesS) in enumerate(list(zip([full_pipline_xgb,full_pipline_rf,full_pipline_mlp],[param_bayesS_xgb,param_bayesS_rf,param_bayesS_mlp]))):
  if iii<=-1:
    best_scores3.append(None)
    bestparams3.append(None)
    continue

  bayes_search = BayesSearchCV(full_pipeline, param_bayesS, n_iter=20, cv=5, verbose=3, n_jobs=2)

  # run the grid search object
  bayes_search.fit(train_X,train_Y)
  val_scores = cross_val_score(bayes_search, test_X, test_Y, cv=5)
  best_scores3.append(bayes_search.best_score_)
  best_scores3.append(val_scores.mean())
  bestparams3.append(bayes_search.best_params_)
  print('-----------------------------------------------')
  print("val_score: %0.2f accuracy with a standard deviation of %0.2f" % (val_scores.mean(), val_scores.std()))
  print('%d___%f___'%(case_study,float(bayes_search.best_score_)) + classifiers_names[iii]+ ': best score {}'.format(bayes_search.best_score_))
  print('%d___%f___'%(case_study,float(bayes_search.best_score_)) + classifiers_names[iii]+': best params {}'.format(bayes_search.best_params_))
  print('-----------------------------------------------')

  pickle.dump(bayes_search,open('%d___%d.pkl'%(iii,case_study),'wb'))
# define ranges for bayes search


     |████████████████████████████████| 101 kB 4.7 MB/s 


Bayesian Result:

---


**RF**
val_score: 0.43 accuracy with a standard deviation of 0.02

3___0.403566___rf: best score 0.4035659718105104

3___0.403566___rf: best params OrderedDict([('my_classifier__max_features', 'auto'), ('my_classifier__min_impurity_decrease', 0.8)])

time:  0:11:35.557117


---



val_score: 0.50 accuracy with a standard deviation of 0.01

3___0.501343___rf: best score 0.5013426384956406

3___0.501343___rf: best params OrderedDict([('my_classifier__max_features', 'auto'), ('my_classifier__min_impurity_decrease', 0.5)])

time:  0:13:28.250860

---
val_score: 0.57 accuracy with a standard deviation of 0.01

3___0.579447___rf: best score 0.5794466100427025

3___0.579447___rf: best params OrderedDict([('my_classifier__max_features', 'auto'), ('my_classifier__min_impurity_decrease', 0.3775439910834993)])

time:  0:14:23.430465

---
val_score: 0.90 accuracy with a standard deviation of 0.00


3___0.920170___rf: best score 0.9201696492597291

3___0.920170___rf: best params OrderedDict([('my_classifier__max_features', 'auto'), ('my_classifier__min_impurity_decrease', 0.0)])

time: time:  0:42:32.451886



---


val_score: 0.90 accuracy with a standard deviation of 0.00

3___0.920229___rf: best score 0.9202292828316108

3___0.920229___rf: best params OrderedDict([('my_classifier__max_features', 'auto'), ('my_classifier__min_impurity_decrease', 0.0)])

time:  0:35:58.411620


---


val_score: 0.90 accuracy with a standard deviation of 0.00

3___0.920318___rf: best score 0.9203176191980185

3___0.920318___rf: best params OrderedDict([('my_classifier__max_features', 'auto'), ('my_classifier__min_impurity_decrease', 0.0), ('my_classifier__min_samples_split', 10)])

time:  1:20:58.912511